<a href="https://colab.research.google.com/github/nyck33/anonymous_face/blob/master/profile_faces_dlib_cnn_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
#install dependencies and wget weights file
! pip3 uninstall opencv_python
! pip3 install dlib argparse opencv_python
#! pip3 uninstall opencv_python
#! pip3 install opencv_python --user
#! pip3 install 'imgaug<0.2.7,>=0.2.5'  --force-reinstall
! wget http://arunponnusamy.com/files/mmod_human_face_detector.dat

Uninstalling opencv-python-4.1.0.25:
  Would remove:
    /root/.local/lib/python3.6/site-packages/cv2/*
    /root/.local/lib/python3.6/site-packages/opencv_python-4.1.0.25.dist-info/*
Proceed (y/n)? y
  Successfully uninstalled opencv-python-4.1.0.25
  Using cached https://files.pythonhosted.org/packages/7b/d2/a2dbf83d4553ca6b3701d91d75e42fe50aea97acdc00652dca515749fb5d/opencv_python-4.1.0.25-cp36-cp36m-manylinux1_x86_64.whl


--2019-07-05 06:06:14--  http://arunponnusamy.com/files/mmod_human_face_detector.dat
Resolving arunponnusamy.com (arunponnusamy.com)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to arunponnusamy.com (arunponnusamy.com)|185.199.108.153|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.arunponnusamy.com/files/mmod_human_face_detector.dat [following]
--2019-07-05 06:06:14--  https://www.arunponnusamy.com/files/mmod_human_face_detector.dat
Resolving www.arunponnusamy.com (www.arunponnusamy.com)... 185.199.111.153, 185.199.109.153, 185.199.110.153, ...
Connecting to www.arunponnusamy.com (www.arunponnusamy.com)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 729940 (713K) [application/octet-stream]
Saving to: ‘mmod_human_face_detector.dat’

mmod_human_face_det 100%[===================>] 712.83K  --.-KB/s    in 0.01s   

2019-07-05 06:06:15 (48.3 MB/s) - ‘mmod_human_face_dete

In [2]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [20]:
#remount drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [50]:
%cd content #/drive/My Drive/Pornflow
!pwd
%cd ../..
%cd content
!pwd

[Errno 2] No such file or directory: 'content #/drive/My Drive/Pornflow'
/
/
/
/content
/content


In [0]:
# import required packages
import cv2
import sys
import dlib
import argparse
import time
import os
import numpy as np
'''
# handle command line arguments##########################################
ap = argparse.ArgumentParser()
ap.add_argument('-i', '--image', required=True, help='path to image file')
ap.add_argument('-w', '--weights', default='./mmod_human_face_detector.dat',
				help='path to weights file')
args = ap.parse_args()

# load input image
image = cv2.imread(args.image)

if image is None:
	print("Could not read input image")
	exit()
#######################################################################
'''
#weights = np.loadtxt('mmod_human_face_detector.dat')

vid_name = "demoPornhub.mp4"
video = cv2.VideoCapture("demoPornhub.mp4")
fourcc = cv2.VideoWriter_fourcc(*'XVID')

if not video.isOpened():  # ファイルがオープンできない場合の処理.
	print("Could not open video")
	sys.exit()

#get vid info
vidw = video.get(cv2.CAP_PROP_FRAME_WIDTH)
vidh = video.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = int(video.get(cv2.CAP_PROP_FPS))

output_video_path = 'demoPorn.avi'

out = cv2.VideoWriter(output_video_path, fourcc, fps, (int(vidw), int(vidh)))

#success, image = vidcap.read()
count = 0

success, image = video.read()

if not success:
	print('Can\'t read video file')
	sys.exit()

while success:

	# initialize hog + svm based face detector
	hog_face_detector = dlib.get_frontal_face_detector()

	# initialize cnn based face detector with the weights
	#cnn_face_detector = dlib.cnn_face_detection_model_v1(args.weights)
	cnn_face_detector= dlib.cnn_face_detection_model_v1('mmod_human_face_detector.dat')

	start = time.time()

	# apply face detection (hog)
	faces_hog = hog_face_detector(image, 1)

	end = time.time()
	#print("Execution Time (in seconds) :")
	#print("HOG : ", format(end - start, '.2f'))

	#loop over detected faces
	for face in faces_hog:
		x = face.left()
		y = face.top()
		w = face.right() - x
		h = face.bottom() - y

		# draw box over face
		cv2.rectangle(image, (x,y), (x+w,y+h), (0,255,0), 2)


	#start = time.time()

	# apply face detection (cnn)
	faces_cnn = cnn_face_detector(image, 1)

	#end = time.time()
	#print("CNN : ", format(end - start, '.2f'))

	# loop over detected faces
	for face in faces_cnn:
		x = face.rect.left()
		y = face.rect.top()
		w = face.rect.right() - x
		h = face.rect.bottom() - y

		# draw box over face
		cv2.rectangle(image, (x,y), (x+w,y+h), (0,0,255), 2)

	# write at the top left corner of the image
	# for color identification
	img_height, img_width = image.shape[:2]
	cv2.putText(image, "HOG", (img_width-50,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
				(0,255,0), 2)
	cv2.putText(image, "CNN", (img_width-50,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
				(0,0,255), 2)


	#display output image
	#cv2.imshow("face detection with dlib", image)
	#cv2.waitKey()

	#save output image 
	cv2.imwrite("cnn_face_detection.png", image)

	out.write(image)
	
	success, image = video.read()
	if not success:
		print('Can\'t read video file')
		sys.exit()

video.release()
out.release()
# close all windows
cv2.destroyAllWindows()